# Uso de la librería `datasets`
Usamos esta librería para cargar de forma dinámica un dataset para entrenar/hacer inferencia en modelos de DL.  
Se instala con:  

`conda install -c huggingface -c conda-forge datasets` 

Ref: https://huggingface.co/docs/datasets/index

Antes de tomar el tiempo para descargar un conjunto de datos, suele ser útil obtener rápidamente información general sobre él. La información de un conjunto de datos se almacena dentro de DatasetInfo y puede incluir detalles como la descripción del conjunto de datos, las características y el tamaño del conjunto de datos.

Utiliza la función `load_dataset_builder()` para cargar un constructor de conjunto de datos y examinar los atributos de un conjunto de datos sin comprometerte a descargarlo:

In [ ]:
from datasets import load_dataset_builder

ds_builder = load_dataset_builder("glue", "mrpc")

In [ ]:
ds_builder

Vemos cuáles son los atributos y métodos del objeto descargado:

In [ ]:
[e for e in dir(ds_builder) if not e.startswith('_')]

In [ ]:
[e for e in dir(ds_builder.info) if not e.startswith('_')]

In [ ]:

#descripción del dataset
print(ds_builder.info.description)

In [ ]:
#características del dataset
ds_builder.info.features

Una división (split) es un subconjunto específico de un conjunto de datos, como entrenamiento y prueba. Enumera los nombres de las divisiones de un conjunto de datos con la función `get_dataset_split_names()`:

In [ ]:
from datasets import get_dataset_split_names

get_dataset_split_names("glue", "mrpc")

Luego puedes cargar una división específica utilizando el parámetro "split". Cargar una división de un conjunto de datos devuelve un objeto Dataset:

In [ ]:
from datasets import load_dataset

dataset = load_dataset("glue", "mrpc", split="train")
dataset

Si no especificas una división, 🤗 Datasets devuelve un objeto DatasetDict en su lugar:

In [ ]:
dataset = load_dataset("glue", "mrpc")
dataset

## Utilizando un dataset
Hay dos tipos de objetos de conjunto de datos, un Dataset regular y luego un **IterableDataset**. Un Dataset proporciona acceso aleatorio rápido a las filas y permite la asignación de memoria para que la carga de conjuntos de datos grandes utilice solo una cantidad relativamente pequeña de memoria del dispositivo. Pero para conjuntos de datos realmente, realmente grandes que ni siquiera caben en el disco o en la memoria, un IterableDataset te permite acceder y utilizar el conjunto de datos sin tener que esperar a que se descargue por completo.

### Objeto `dataset`
Cuando cargas una división de un conjunto de datos, obtienes un objeto Dataset. Puedes hacer muchas cosas con un objeto Dataset, por lo que es importante aprender cómo manipular e interactuar con los datos almacenados en su interior.

In [ ]:
dataset = load_dataset("glue", "mrpc", split="train")

In [ ]:
[e for e in dir(dataset) if not e.startswith('_')]

In [ ]:
dataset.version

In [ ]:
dataset.features


#### Indexación

Un Dataset contiene columnas de datos, y cada columna puede ser un tipo de dato diferente. El índice, o etiqueta de eje, se utiliza para acceder a ejemplos del conjunto de datos. Por ejemplo, la indexación por fila devuelve un diccionario con un ejemplo del conjunto de datos.

In [ ]:
#primera fila del dataset
dataset[0]

In [ ]:
#última fila del dataset
dataset[-1]

La indexación por el nombre de la columna devuelve una lista de todos los valores en esa columna:

In [ ]:
dataset['sentence1']

Puedes combinar la indexación por fila y por nombre de columna para obtener un valor específico en una posición determinada:

In [ ]:
dataset[0]["sentence1"]

Al revés también funciona pero es más lento:

In [ ]:
dataset["sentence1"][0]

#### Slicing
El uso de la técnica de "slicing" devuelve una rebanada o subconjunto del conjunto de datos, lo cual es útil para ver varias filas a la vez. Para hacer un corte (slice) de un conjunto de datos, utiliza el operador ":" para especificar un rango de posiciones.

In [ ]:
#filas entre la 3 y la 6
dataset[3:6]

In [ ]:
dataset

### Objeto `IterableDataset`

Un IterableDataset se carga cuando estableces el parámetro "streaming" en True al cargar un conjunto de datos utilizando la función `load_dataset()`:

In [ ]:
iterable_dataset = load_dataset("rotten_tomatoes", split="train", streaming=True)
for example in iterable_dataset:
     print(example)
     break

In [ ]:
iterable_dataset

In [ ]:
iterable_dataset.features

In [ ]:
iterable_dataset.dataset_size

Un IterableDataset itera progresivamente sobre un conjunto de datos de un ejemplo a la vez, por lo que no es necesario esperar a que todo el conjunto de datos se descargue antes de poder usarlo. Como puedes imaginar, esto es muy útil para conjuntos de datos grandes que deseas utilizar de inmediato.

Sin embargo, esto significa que el comportamiento de un IterableDataset es diferente al de un Dataset regular. No obtienes acceso aleatorio a los ejemplos en un IterableDataset. En su lugar, debes iterar sobre sus elementos, por ejemplo, llamando a `next(iter())` o utilizando un bucle for para obtener el siguiente elemento del IterableDataset.

In [ ]:
next(iter(iterable_dataset))

Puedes obtener un subconjunto del conjunto de datos con un número específico de ejemplos utilizando la función IterableDataset.take():

In [ ]:
list(iterable_dataset.take(3))

## Preprocesamiento

Además de cargar conjuntos de datos, el principal objetivo de 🤗 Datasets es ofrecer una amplia variedad de funciones de preprocesamiento para llevar un conjunto de datos a un formato adecuado para el entrenamiento con tu marco de aprendizaje automático.

Hay muchas formas posibles de preprocesar un conjunto de datos, y todo depende de tu conjunto de datos específico. A veces es posible que necesites cambiar el nombre de una columna, y otras veces puede que necesites desenrollar campos anidados. 🤗 Datasets ofrece una forma de hacer la mayoría de estas cosas. Pero en casi todos los casos de preprocesamiento necesitarás tokenizar un conjunto de datos de texto.

El último paso de preprocesamiento generalmente implica establecer el formato del conjunto de datos para que sea compatible con el formato de entrada esperado por tu marco de aprendizaje automático.
### Tokenizado
Cargamos un modelo de tokenizado de la librería 🤗 Transformers y ejecutamos

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
dataset = load_dataset("rotten_tomatoes", split="train")

In [ ]:
dataset

In [ ]:
tokenizer(dataset[0]["text"])

La forma más rápida de tokenizar todo tu conjunto de datos es utilizar la función `map()`. Esta función acelera la tokenización al aplicar el tokenizador a lotes de ejemplos en lugar de ejemplos individuales. Establece el parámetro batched en True:

In [ ]:
def tokenization(example):
    return tokenizer(example["text"], truncation=True, padding=True)

dataset = dataset.map(tokenization, batched=True)

In [ ]:
dataset

In [ ]:
dataset[0]

Establece el formato de tu conjunto de datos para que sea compatible con tu framework de aprendizaje automático:

Utiliza la función `set_format()` para convertir el formato del dataset a torch y especificar las columnas a formatear. Esta función aplica el formato on-the-fly. Después de convertir a tensores PyTorch, se convierte en un objeto `torch.utils.data.DataLoader`:

In [ ]:
import torch

dataset.set_format(type="torch", columns=["input_ids", "token_type_ids", "attention_mask", "label"])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32)

Utiliza la función `to_tf_dataset()` para establecer el formato del conjunto de datos para que sea compatible con TensorFlow. También necesitarás importar un objeto de la clase `DataCollator` de 🤗 Transformers para combinar las longitudes de secuencia variables en un solo lote con longitudes iguales:

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")
tf_dataset = dataset.to_tf_dataset(
    columns=["input_ids", "token_type_ids", "attention_mask"],
    label_cols=["labels"],
    batch_size=2,
    collate_fn=data_collator,
    shuffle=True
)

El dataset creado se puede usar en Tensorflow directamente para entrenar/hacer inferencia en un modelo:
```python
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=callbacks)
```

## Evaluar las predicciones
🤗 Datasets proporciona varias métricas comunes y específicas de procesamiento del lenguaje natural (NLP) para medir el rendimiento de tus modelos. vamos a ver cómo cargar una métrica y la utilizarla para evaluar las predicciones de tu modelo.

Puedes ver qué métricas están disponibles con la función `list_metrics()`:

In [ ]:
from datasets import list_metrics
metrics_list = list_metrics()
len(metrics_list)


In [ ]:
print(metrics_list)

Es muy fácil cargar una métrica con 🤗 Datasets. De hecho, te darás cuenta de que es muy similar a cargar un conjunto de datos. Puedes cargar una métrica desde el Hub con la función load_metric():

In [ ]:
from datasets import load_metric
metric = load_metric('glue', 'mrpc')

### Objeto `Metric`
Antes de comenzar a utilizar un objeto Metric, es importante conocerlo un poco mejor. Al igual que con un conjunto de datos, puedes obtener información básica sobre una métrica. Por ejemplo, accedemos al parámetro `inputs_description` en `datasets.MetricInfo` para obtener más información sobre el formato de entrada esperado de una métrica y algunos ejemplos de uso:

In [ ]:
print(metric.inputs_description)

### Calcular las métricas
Una vez que hayas cargado una métrica, estás listo para usarla para evaluar las predicciones de un modelo. Proporciona las predicciones del modelo y las referencias a la función `compute()`:
```python
model_predictions = model(model_inputs)
final_score = metric.compute(predictions=model_predictions, references=gold_references)
```

## Creación de un dataset
A veces, es posible que necesites crear un conjunto de datos si estás trabajando con tus propios datos. Crear un conjunto de datos con 🤗 Datasets te brinda todas las ventajas de la biblioteca: carga y procesamiento rápido, capacidad de trabajar con conjuntos de datos enormes, asignación de memoria y más. Puedes crear fácil y rápidamente un conjunto de datos con enfoques de bajo código de 🤗 Datasets, lo que reduce el tiempo necesario para comenzar a entrenar un modelo.

### Creación a partir de archivos locales
Puedes crear un conjunto de datos a partir de archivos locales especificando la ruta a los archivos de datos. Hay dos formas de crear un conjunto de datos utilizando los métodos from_:

El método `from_generator()` es la forma más eficiente en términos de memoria para crear un conjunto de datos a partir de un generador debido al comportamiento iterativo de los generadores. Esto es especialmente útil cuando trabajas con un conjunto de datos realmente grande que puede no caber en memoria, ya que el conjunto de datos se genera progresivamente en disco y luego se asigna a memoria.

In [ ]:
from datasets import Dataset
def gen():
    yield {"pokemon": "bulbasaur", "type": "grass"}
    yield {"pokemon": "squirtle", "type": "water"}
ds = Dataset.from_generator(gen)
ds[0]

In [ ]:
ds

Un `IterableDataset` basado en generador necesita ser iterado con un bucle `for`, por ejemplo:

In [ ]:
from datasets import IterableDataset
ds = IterableDataset.from_generator(gen)
for example in ds:
    print(example)

In [ ]:
ds

El método `from_dict()` es una forma directa de crear un dataset a partir de un diccionario:

In [ ]:
from datasets import Dataset
ds = Dataset.from_dict({"pokemon": ["bulbasaur", "squirtle"], "type": ["grass", "water"]})
ds[0]

In [ ]:
ds

Un ejemplo completo del uso de un dataset en una tarea de clasificación se puede encontrar en https://huggingface.co/docs/transformers/tasks/sequence_classification

### Ejemplo
Creamos un dataset a partir de un objeto generador que devuelve los documentos de un archivo de texto línea a línea.

In [ ]:
def build_texts(fname):
    with open(fname) as f:
        for line in f:
            yield ({'texto': line, 'long': len(line)})



In [ ]:
textos = build_texts("lee_background.cor")

In [ ]:
for t in textos:
    print(t)
    break

In [ ]:
ds = Dataset.from_generator(build_texts, gen_kwargs={"fname": 'lee_background.cor'})

In [ ]:
ds

Este dataset en realidad también se podría haber creado con el método `from_file` de la librería:

In [ ]:
from datasets import load_dataset

ds = load_dataset('text', data_files = {'train': 'lee_background.cor'}, encoding='ISO-8859-1')

In [ ]:
ds

In [ ]:
ds['train'][0]